# Basic preparations

In this Notebook, the geospatial fabric for the "Nelson Churchill River Basin" gauge is extracted from the `MERIT-Basins` dataset.

If you are using Graham HPC, and have access to Clark's Research Group allocation (`rrg-mclark`), you may find `MERIT-Basins` layers under the following path:</br>
`/project/rrg-mclark/data/geospatial-data/MERIT-Basins/`

If you are using Graham HPC, and have access to Global Water Futures Observatories allocation (`rpp-kshook`), you may find `MERIT-Basins` layers under the following path:</br>
`/project/rpp-kshook/Climate_Forcing_Data/geospatial-data/MERIT-Basins`

The version used is `MERIT_Hydro_v07_Basins_v01_bugfix1` which is a directory under the root directory of the dataset.

Let's get started with our workflow and import necessary Python libraries:

In [ ]:
import geopandas as gpd # version 0.14.0

In [ ]:
import pandas as pd # version 1.4.0
import numpy as np # version 1.22.2
import matplotlib.pyplot as plt # version 3.5.1
from shapely.geometry import Point # version 2.0.1
import hydrant.topology.geom as gm # version 0.1.0-dev1
import subprocess # built-in Python 3.10.2
import os # built-in Python 3.10.2
import glob # built-in Python 3.10.2

`Hydrant` is important in this Notebook. We are only using the `topology.geom`etry module.

Path definitions (system dependant - modify accordingly):

In [ ]:
# geofabric paths
merit_basins_root_path = '/project/6008034/Climate_Forcing_Data/geospatial-data/MERIT-Basins/MERIT_Hydro_v07_Basins_v01_bugfix1'
merit_basins_geom_path = os.path.join(merit_basins_root_path, 'pfaf_level_02')
merit_basins_nca_path = os.path.join(merit_basins_root_path, 'coastal_hillslopes')
# output paths
output_path = '/home/fuaday/github-repos/MESH-Nelson-Churchill-Basin-Vector/1-geofabric/ncrb-geofabric/'
bs_path = '/home/fuaday/github-repos/MESH-Nelson-Churchill-Basin-Vector/NCRB_shapefiles/NCRB_SubbasinsDissolved.shp'
nctr_test = '/home/fuaday/github-repos/MESH-Nelson-Churchill-Basin-Vector/NCRB_shapefiles/non.shp' #ncontr_test.shp
nctr_testout = '/home/fuaday/github-repos/MESH-Nelson-Churchill-Basin-Vector/NCRB_shapefiles/ncontr_test2.shp'

In [ ]:
ncrb = pd.concat([gpd.read_file(f).to_crs(epsg=4326) for f in glob.glob(bs_path)])

In [ ]:
files = glob.glob(nctr_test)

# Load files, set CRS, and concatenate
gdfs = []
for f in files:
    gdf = gpd.read_file(f)
    if gdf.crs is None:
        gdf.set_crs(epsg=4326, inplace=True)
    gdfs.append(gdf.to_crs(epsg=4326))
ncrb_nca = pd.concat(gdfs)

In [ ]:
print(ncrb.columns)

In [ ]:
ncrb_nca.plot()

# Reading `MERIT-Basins` Geospatial Fabric Dataset

Upon **visual** inspection (you may use `QGIS` or similar programs), layer #71 has been identified to include necessary sub-basins for ncrb

As you may see in the cell below, we are using Python `list`s to enable reading multiple layers at once. There are cases where a basin of interest is shared between multiple `pfaf` layers.

For now, let's read the files, one by one:

In [ ]:
# file names
# catchments (subbasin)
cat_files = [
    'cat_pfaf_71_MERIT_Hydro_v07_Basins_v01_bugfix1.shp',
]
# rivers (river segments)
riv_files = [
    'riv_pfaf_71_MERIT_Hydro_v07_Basins_v01_bugfix1.shp',
]
# non-contributing catchments (those without any river segments defined for them)
nca_files = [
    'hillslope_71_clean.shp',
]

# reading in data in an iterative manner
cat = pd.concat([gpd.read_file(os.path.join(merit_basins_geom_path, f)) for f in cat_files])
riv = pd.concat([gpd.read_file(os.path.join(merit_basins_geom_path, f)) for f in riv_files])
nca = pd.concat([gpd.read_file(os.path.join(merit_basins_nca_path, f)) for f in nca_files])

Since `MERIT-Basins` layers do not come with correct Coordinate Reference System (CRS) information, we need to specify this manually. The `EPSG` code for the `MERIT-Basins` layer is `4326`. Please refer to the following for more information: https://en.wikipedia.org/wiki/EPSG_Geodetic_Parameter_Dataset

In [ ]:
# specifying epsg:4326 for all the MERIT-Basins layers
cat.set_crs(epsg=4326, inplace=True)
nca.set_crs(epsg=4326, inplace=True)
riv.set_crs(epsg=4326, inplace=True)

# Show the EPSG of all geospatial layers
print(f'`cat` CRS: {cat.crs}')
print(f'`riv` CRS: {riv.crs}')
print(f'`nca` CRS: {nca.crs}')

In [ ]:
# Printing the sizes
print("Size of catchments (cat):", cat.shape)
print("Size of rivers (riv):", riv.shape)
print("Size of non-contributing areas (nca):", nca.shape)

In [ ]:
# Display the headers of each DataFrame
print("Header for Catchments (cat):")
print(cat.head())

print("\nHeader for Rivers (riv):")
print(riv.head())

print("\nHeader for Non-Contributing Areas (nca):")
print(nca.head())

# Preparing `cat`, `riv`, and `nca` objects for `NCRB`

## Preparing `MERIT-Basins` Layers

Before subsetting the entire layer #71 of the `MERIT-Basins` dataset, we have to assure the layers are ready to be further processed by the `Hydrant` package. Fortunately, `Hydrant` provides necessary functionalities to work with this specific geospatial fabric (applicable to any geospatial fabric in reality). 

In doing so, Hydrant's `geom` module provides the `prepare_cat(...)` function to prepare the `MERIT-Basins` geosptial fabric's sub-basins (or catchments) for the next post-processing steps. Please note that since the non-contributing areas (`nca`) are technically considered sub-basins, they are taken care of using this functionality of `Hydrant`:

In [ ]:
# Hydrant's `geom` module provides the `prepare_cat`
# function to prepare the `MERIT-Basins` geosptial
# fabric for next post-processing steps

catchments = gm.prepare_cat(
    cat=cat, # 
    cat_col_id='COMID',
    cst=nca,
    cst_col_mapper={'FID':'COMID'},
    cst_col_id='COMID'
)

# You may see the "docstring" of the `gm.prepare_cat`
# function by running:
# >>> gm.prepare_cat?
# in a separate Jupyter cell (without the >>>), or by
# running simply:
# >>> print(gm.prepare_cat.__doc__)

In [ ]:
# added from smm example

# first, creating the directory
try:
    os.makedirs(output_path)
except FileExistsError:
    pass
catchments.to_file(os.path.join(output_path, 'NCRB_catchmentsTest.shp'))

Similarly, the `geom` module provides the `prepare_riv(...)` function to prepare the `MERIT-Basins` geospatial fabric's river segments for the next post-processing steps:

In [ ]:
# Similarly, the `geom` module provides the
# `prepare_riv` function to prepare the `MERIT-Basins`
# geospatial fabric's river segments for the next
# post-processing steps:

rivers = gm.prepare_riv(
    riv=riv,
    riv_cols={
        'id':'COMID',
        'next_id':'NextDownID',
        'slope':'slope',
        'length':'lengthkm',
        'length_direct':'lengthdir'
    },
    cat=catchments,
    cat_cols={
        'id':'COMID',
        'hillslope':'hillslope',
        'geom':'geometry'
    }
)

# You may see the "docstring" of the `gm.prepare_riv`
# function by running:
# >>> gm.prepare_riv?
# in a separate Jupyter cell (without the >>>), or by
# running simply:
# >>> print(gm.prepare_riv.__doc__)

In Python, you may always access the "docstring" documentations for the functions and classes by running:
```python
>>> print(func.__doc__)
```
Or, if you are working in the Jupyter environment, you may simply run the following in a separate Jupyter cell:
```ipython
[ln1] func?
```
If you are interested in reading up on the functionality of each function used above, use the mentioned methods to print the "docstrings".

Therefore, if you would like to read up on the `gm.prepare_cat(...)` or `gm.prepare_riv(...)` functionality, simply uncomment and execute the following cell:

In [ ]:
modified_gdf = gm.flag_ncaalg(catchments, ncrb_nca,0.3, nctr_testout)

In [ ]:
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch

# Adjusting global font sizes using rcParams for the plot
plt.rcParams.update({'font.size': 14})

# Assuming 'ncrb_catchments' is your GeoDataFrame loaded previously
# ncrb_catchments = gpd.read_file('path_to_your_shapefile/ncrb_catchments.shp')

# Calculating counts
count_1 = ncrb_catchments[ncrb_catchments['ncontr'] == 1].shape[0]
count_2 = ncrb_catchments[ncrb_catchments['ncontr'] == 2].shape[0]

# Custom colors for the unique values in 'ncontr', assuming they are 1 and 2
colors = ['#1f77b4', '#ff7f0e']  # Blue for 1, Orange for 2
cmap = ListedColormap(colors)

# Plotting
fig, ax = plt.subplots(1, 1, figsize=(20, 12))
ncrb_catchments.plot(column='ncontr', ax=ax, cmap=cmap, edgecolor=None)

# Create a legend with larger font
legend_handles = [Patch(color=value_colors[value], label=f'{label} (n={count})') for value, label, count in zip([1, 2], ['1', '2'], [count_1, count_2])]
legend = ax.legend(handles=legend_handles, title="NCONTR values", fontsize=14)
plt.setp(legend.get_title(), fontsize=14)

# Adding text to the plot with counts
text_str = f'Count of 1: {count_1}\nCount of 2: {count_2}'
ax.text(0.05, 0.95, text_str, transform=ax.transAxes, fontsize=14,
        verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))

# Save the figure
plt.savefig('ncrb_catchments_plot.png', dpi=300, bbox_inches='tight')

In [ ]:
ncrb_catchments[ncrb_catchments['ncontr'] == 1].shape[0]
ncrb_catchments[ncrb_catchments['ncontr'] == 2].shape[0]

In [ ]:
#gm.prepare_cat?

In [ ]:
# gm.prepare_riv?

In [ ]:
# ADDED from smm example
ncrb_catchments, ncrb_rivers = gm.intersect_topology(
    cat=modified_gdf,
    cat_cols={
        'id':'COMID'
    },
    riv=rivers,
    riv_cols={
        'id':'COMID',
        'next_id':'NextDownID'
    },
    shapefile=ncrb)

In [ ]:
# # ADDED from smm example
# ncrb_catchments, ncrb_rivers = gm.intersect_topology(
#     cat=catchments,
#     cat_cols={
#         'id':'COMID'
#     },
#     riv=rivers,
#     riv_cols={
#         'id':'COMID',
#         'next_id':'NextDownID'
#     },
#     shapefile=ncrb)

In [ ]:
ncrb_catchments

In [ ]:
ax = ncrb.plot(color='black', alpha=0.3, figsize=(10, 20))
#ncrb_rivers.plot(ax=ax, color='red')
ncrb_catchments.plot(ax=ax)

Now, let's plot what have extracted from the larger `MERIT-Basins` geospatial fabric:

In [ ]:
fig, ax = plt.subplots(
    nrows=1,
    ncols=1,
    figsize=(20, 12)
)

# Assuming 'ncrb_catchments' and 'ncrb_rivers' are GeoDataFrames and 'point' is a geometry point for the gauge location

# sub-basins
ncrb_catchments.plot(ax=ax, color='gray', edgecolor='black', alpha=0.8, linewidth=0.3, zorder=1)  # Adjust `linewidth` as needed
# river segments
ncrb_rivers.plot(ax=ax, color='blue', alpha=1, linewidth=0.3, zorder=2)  # Adjust `linewidth` as needed for rivers

# gauge location
# Assuming 'point' is defined somewhere above as a geometry point
# ax.scatter(point.x, point.y, color='red', alpha=0.8, s=100, zorder=3)  # Use `s` for size in scatter plot, adjust as needed

plt.show()


# Saving Extracted Geospatial Fabric upstream of Bow River at Banff Hydrometric Station

In [ ]:
# saving the results into the `output_path` directory

# first, creating the directory
try:
    os.makedirs(output_path)
except FileExistsError:
    pass

# then, saving the data
ncrb_catchments.to_file(os.path.join(output_path, 'ncrb_subbasins.shp'))
ncrb_rivers.to_file(os.path.join(output_path, 'ncrb_rivers.shp'))

___

If you face any issues, e-mail [kasra.keshavarz1 AT ucalgary DOT ca](mailto:kasra.keshavarz1@ucalgary.ca).

In [ ]:
# import geopandas as gpd

# def assign_intersection_flag_gdf_sindex(
#     gdf1: gpd.GeoDataFrame,
#     gdf2: gpd.GeoDataFrame,
#     threshold: float = 0.1,  # Threshold set to 10% by default
#     output_path: str = None
# ) -> gpd.GeoDataFrame:
#     """
#     Efficiently assigns a flag to polygons in the first GeoDataFrame based on significant intersection
#     (defined by a threshold) with polygons in the second GeoDataFrame using spatial indexing.

#     Parameters
#     ----------
#     gdf1 : gpd.GeoDataFrame
#         The first GeoDataFrame.
#     gdf2 : gpd.GeoDataFrame
#         The second GeoDataFrame.
#     threshold : float, optional
#         The threshold for considering an intersection significant, as a fraction of
#         the first GeoDataFrame's polygon area (default is 0.1 for 10%).
#     output_path : str, optional
#         Path where the modified first GeoDataFrame should be saved. If None, the file is not saved.

#     Returns
#     -------
#     gpd.GeoDataFrame
#         The modified GeoDataFrame of the first GeoDataFrame with the 'ncontr' column added.
#     """
#     # Initialize the 'ncontr' column to 0 for all rows in gdf1
#     gdf1['ncontr'] = 0
    
#     # Create spatial index for gdf2
#     spatial_index = gdf2.sindex
    
#     # Iterate over gdf1 using spatial indexing to find potential intersections
#     for index, row in gdf1.iterrows():
#         # Use spatial index to find potential intersections
#         possible_matches_index = list(spatial_index.query(row['geometry'], predicate='intersects'))
#         if not possible_matches_index:
#             continue  # No intersections, move to next row
        
#         # Filter possible matches for actual intersection
#         possible_matches = gdf2.iloc[possible_matches_index]
#         actual_intersections = possible_matches[possible_matches.intersects(row['geometry'])]
        
#         # Calculate area fractions for actual intersections
#         for _, match in actual_intersections.iterrows():
#             intersection_area = row['geometry'].intersection(match['geometry']).area
#             area_fraction = intersection_area / row['geometry'].area
#             if area_fraction > threshold:
#                 gdf1.at[index, 'ncontr'] = 1
#                 break  # Break after finding the first significant intersection
    
#     # Save the modified gdf1 to a new shapefile if an output path is provided
#     if output_path is not None:
#         gdf1.to_file(output_path)
    
#     return gdf1